## Keras for Geometry dash

This time we are going to use Keras-RL for the same project

In [5]:
# Lets see if CUDA is available
import tensorflow as tf

if tf.test.is_built_with_cuda():
    print("TensorFlow was built with CUDA")
else:
    print("TensorFlow was not built with CUDA")

print("GPUs available: ", tf.config.list_physical_devices('GPU'))

TensorFlow was built with CUDA
GPUs available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
import cv2
import numpy as np
import pyautogui

global green_per
green_per = 0.1
def capture_screen(region=(0, 0, 640, 480), resize_dim=(160, 120)):
    global green_per

    screen = pyautogui.screenshot(region=region)
    
    # Convert screenshot into numpy array
    frame = np.array(screen, dtype=np.uint8)
    
    # Convert to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Convert to HSV
    frame_hsv = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2HSV)
    
        # Define range of green color in HSV
    lower_green = np.array([50, 100, 128])
    upper_green = np.array([70, 255, 255])
    # Adjust these values according to your needs
    
    # Threshold the HSV image to get only green colors
    mask = cv2.inRange(frame_hsv, lower_green, upper_green)
    
    # Calculate the percentage of green pixels
    green_per = np.sum(mask == 255) / (mask.size) * 100
    
    # Resize image
    frame_resized = cv2.resize(frame_rgb, resize_dim)
    
    return frame_resized

## Create a Gym Enviroment
---

In [7]:
import gym
from gym import spaces
import cv2
import numpy as np
import pyautogui
q
class GeometryDashEnv(gym.Env):
    def __init__(self):
        super(GeometryDashEnv, self).__init__()
        # Define el espacio de observación (estado del juego)
        self.observation_space = spaces.Box(low=0, high=255, shape=(160, 120, 3), dtype=np.uint8)
        
        # Define el espacio de acciones
        self.action_space = spaces.Discrete(2)  # 0: do nothing, 1: jump
        
        # Inicializa el porcentaje de verde (para detectar si el cubo está vivo)
        self.green_per = 0.1
    

    def capture_screen(self, region=(0, 0, 640, 480), resize_dim=(160, 120)):
        screen = pyautogui.screenshot(region=region)
        frame = np.array(screen, dtype=np.uint8)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convertir a HSV y encontrar el porcentaje de verde
        frame_hsv = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2HSV)
        lower_green = np.array([50, 100, 128])
        upper_green = np.array([70, 255, 255])
        mask = cv2.inRange(frame_hsv, lower_green, upper_green)
        self.green_per = np.sum(mask == 255) / mask.size * 100
        
        # Redimensionar imagen
        frame_resized = cv2.resize(frame_rgb, resize_dim)
        return frame_resized

    def reset(self):
        return self.capture_screen()

    def step(self, action):
        # Realizar acción
        if action == 1:
            pyautogui.click()
        
        # Capturar el nuevo estado
        self.state = self.capture_screen()
        
        done = False
        reward = 1  # Recompensa por seguir vivo
        
        # Si el porcentaje de verde es muy bajo, podría indicar que el cubo ha muerto
        if self.green_per == 0: 
            done = True
            reward = -50   # Penalización por morir
        
        info = {'green_percentage': self.green_per}
        return self.state, reward, done, info

    def render(self, mode='human'):
        display_img = self.state.copy()  # Haz una copia para no modificar el estado original
        
        if mode == 'human':
            # Dibuja texto con el porcentaje de verde detectado
            cv2.putText(display_img, f'Green %: {self.green_per:.2f}', (10, 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            cv2.imshow('Gameplay with Stats', display_img)
            cv2.waitKey(1)
        elif mode == 'rgb_array':
            return display_img



In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger


num_actions = 2 # jump or not jump
path = './model.keras'

env = GeometryDashEnv()
input_shape = (120, 160, 3)
# Define your CNN model
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(num_actions, activation='linear'))

# Define your RL agent
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=10000, window_length=1)
dqn = DQNAgent(model=cnn_model, nb_actions=num_actions, memory=memory, nb_steps_warmup=1000,
               target_model_update=1e-2, policy=policy)

# Compile the agent
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Train the agent
dqn.fit(env, nb_steps=10000, visualize=True, verbose=1)

# Test the trained agent
dqn.test(env, nb_episodes=5, visualize=True)

Training for 10000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected conv2d_2_input to have 4 dimensions, but got array with shape (1, 1, 120, 160, 3)